In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

We want to find $sin(x)$ for any $x\in [0,2\pi]$

To evaluate $sin$, we follow three approaches:
- using the expectation value of a single qubit
- using binary fractional approximation 
- Quantum models as Fourier series (https://pennylane.ai/qml/demos/tutorial_expressivity_fourier_series/)


In [2]:
np.random.seed(0)
# We use random data points with random spacing to make sure 
#   our model understands the behavior of sin everywhere
num_p = 100 # number of data points
features = np.random.uniform(0, 2 * np.pi, num_p)
labels = np.sin(features)

# Shuffle and split the data into training and test sets
np.random.seed(42)
indices = np.random.permutation(num_p)
split_index = int(num_p * 0.8) # 80% for training
train_indices, test_indices = indices[:split_index], indices[split_index:]
features_train = features[train_indices]
features_test = features[test_indices]
labels_train = labels[train_indices]
labels_test = labels[test_indices]

# Expectation value encoding $-$ single qubit 

### Input:
`state_preparation` rely on one dimensional feature $x$. So, we only need one qubit. Since $x\in [0,2\pi]$, we apply one rotaion in `state_preparation`.

In [3]:
num_qubits = 1
dev = qml.device("default.qubit", wires=num_qubits)

In [4]:
# Define the state preparation function
def state_preparation(feature):
    qml.RY(feature, wires=0)
    
# Define the variational layer    
def layer(layer_weights):
    num_qubits = 1
    for i in range(num_qubits):
        qml.Rot(*layer_weights[i], wires=i)

# Define the variational circuit
@qml.qnode(dev)
def circuit(weights, feature):
    state_preparation(feature)
    for layer_weights in weights:
        layer(layer_weights)
    return qml.expval(qml.PauliZ(0))


### The meaning of our output
We take the expectation value as the solution, and we don't assume it to be $\{-1,1\}$ but in the range $[-1,1]$. In real life, this means that after creating the model, to use it, we run our circuit many times to approximate the solution. 

With our approach, we have to modify our `accuracy` function since we don't expect deterministic results. So, we set a `precision` parameter.

In [5]:
def square_loss(labels, predictions):
    return np.mean((labels - qml.math.stack(predictions)) ** 2)

def variational_classifier(weights, bias, features):
    prediction = np.array(circuit(weights, features))
    return prediction + bias

def cost(weights, bias, features, labels):
    predictions = np.array([variational_classifier(weights, bias, feature) for feature in features], requires_grad=True)
    return square_loss(labels, predictions)

def accuracy(labels, predictions, precision=0.001):
    acc = sum(abs(l - p) < precision for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc

In [6]:
num_qubits = 1
num_layers = 4
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.5, requires_grad=True)

In [7]:
opt = NesterovMomentumOptimizer(0.005)
batch_size = 10
weights = weights_init
bias = bias_init
for it in range(150):
    # Update the weights by one optimizer step, using only a limited batch of data
    batch_index = np.random.randint(0, len(features_train), (batch_size,))
    features_batch = features_train[batch_index]
    labels_batch = labels_train[batch_index]
    weights, bias = opt.step(cost, weights, bias, features=features_batch, labels=labels_batch)
    # Compute accuracy
    predictions = [(variational_classifier(weights, bias, x)) for x in features_train]
    current_cost = cost(weights, bias, features_train, labels_train)
    acc = accuracy(labels_train, predictions, precision=0.001)

    print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")

Iter:    1 | Cost: 1.0655053 | Accuracy: 0.0000000
Iter:    2 | Cost: 1.0347397 | Accuracy: 0.0000000
Iter:    3 | Cost: 0.9990782 | Accuracy: 0.0000000
Iter:    4 | Cost: 0.9491864 | Accuracy: 0.0000000
Iter:    5 | Cost: 0.8835238 | Accuracy: 0.0000000
Iter:    6 | Cost: 0.8146962 | Accuracy: 0.0000000
Iter:    7 | Cost: 0.7453175 | Accuracy: 0.0000000
Iter:    8 | Cost: 0.6687105 | Accuracy: 0.0000000
Iter:    9 | Cost: 0.5850933 | Accuracy: 0.0125000
Iter:   10 | Cost: 0.5051143 | Accuracy: 0.0000000
Iter:   11 | Cost: 0.4241238 | Accuracy: 0.0000000
Iter:   12 | Cost: 0.3472919 | Accuracy: 0.0000000
Iter:   13 | Cost: 0.2752557 | Accuracy: 0.0000000
Iter:   14 | Cost: 0.2097363 | Accuracy: 0.0000000
Iter:   15 | Cost: 0.1537995 | Accuracy: 0.0000000
Iter:   16 | Cost: 0.1090033 | Accuracy: 0.0000000
Iter:   17 | Cost: 0.0714791 | Accuracy: 0.0000000
Iter:   18 | Cost: 0.0429970 | Accuracy: 0.0000000
Iter:   19 | Cost: 0.0222916 | Accuracy: 0.0000000
Iter:   20 | Cost: 0.0089702 | 

In [8]:
predictions = [(variational_classifier(weights, bias, x)) for x in features_test]
acc = accuracy(labels_test, predictions)
print("Accuracy on test data:", f"{acc:0.2f}")


Accuracy on test data: 1.00


We achieved 100% accuracy with $0.001\$ precision on on unseen data. We can use higher precision, but this requires carefully tuning the model parameters. Also, for more digits, using the model will be more expensive since then, we have to run the model many more times to get results within the desired precision.

The main issue with this model is that it's not efficient to use. For extreme precision results, we have to run the circuit many times until our solution approaches a value withing our expected range.

# Binary fraction extraction $-$ multi qubit

The produced model from this approach encodes the solution in a binary string with `n` binary fractions. This means we measure more than one qubit. 

$sin(x) = \alpha$ where in binary $\alpha = a_0 . a_1a_2...a_n$

$$\alpha = a_0 2^0 + a_1 2^{-1} + a_2 2^{-2} + ... + a_n 2^{-n}$$
$a_i\in \{0, 1\}$

Our goal is to design a model that accepts any given $n$.

*Note: we might get away with one qubit and get the same results as if we have $n$ qubits if we find a way to recycle our qubit, similar to what is done in some phase estimation techniques.*


To implement this approach, I can think of two main paths:
1) Get the first n binary fractional digits with no approximation
2) Get an approximation that has n fractional digits

To see the importance of the difference, let's take $0.93$ as an example. In binary, the first four digits are: $$0.1110$$  which is $0.875$ in decimal. But the binary fraction $0.1111$ is $0.9375$ in decimal.

I will be training my model to get the former approximation. In our `cost` function, I will use the difference between the generated values by our quantum circuit with the first `n` digits of the actual answer. This means that we don't really need to know the ideal binary approximation. 

*Note: my approach assumes that the accuracy of the decimal values are higher than the accuracy we intend to generate using `n` binary fractional digits.*


In [9]:
np.random.seed(0)
# We use random data points with random spacing to make sure 
#   our model understands the behavior of sin everywhere
num_p = 300 # number of data points
features = np.random.uniform(0, 2 * np.pi, num_p)
labels = np.sin(features)

# Shuffle and split the data into training and test sets
np.random.seed(42)
indices = np.random.permutation(num_p)
split_index = int(num_p * 0.8) # 80% for training
train_indices, test_indices = indices[:split_index], indices[split_index:]
features_train = features[train_indices]
features_test = features[test_indices]
labels_train = labels[train_indices]
labels_test = labels[test_indices]

In [10]:
n = 3 # number of fractional binary digits
num_qubits = (n + 2) # qubit for sign and qubit for first integer digit
dev = qml.device("default.qubit", wires=num_qubits)

I encode the data using a simple format:

[sign, integer part of the binary number, `n` binary fractions to represent bits]

In [11]:
# Generate the first n binary fractional digits with their sign 
# Output format [sign: {1 for + | -1 for -}, int digit: {1 for one | -1 for zero}, 
#                       fractional n digits: {1 for one | -1 for zero} ...]
float_to_binary = lambda num, precision: (
    [1 if num >= 0 else -1] + # sign
    [1 if num == 1. else -1] +  # 1 if bit = 1, -1 if bit = 0
        [1 if (num := (abs(num) % 1) * 2) >= 1 else -1 for _ in range(precision)] 
        )
# Inverses the previous function to get decimal values
binary_to_float = lambda binary_rep: (1 if binary_rep[0] == 1 else -1) * sum((bit == 1) * 2**(-i) for i, bit in enumerate(binary_rep[2:], start=1))


features_train2 = np.array([float_to_binary(feature, n) for feature in features_train])
features_test2 = np.array([float_to_binary(feature, n) for feature in features_test])
# Labels are generated using the binary representation of the input
labels_train2 = np.array( [float_to_binary(feature, n)for feature in  
                           [np.sin(binary_to_float(i)) for i in features_train2]
                           ] 
                         )
labels_test2 = np.array( [float_to_binary(feature, n)for feature in  
                           [np.sin(binary_to_float(i)) for i in features_test2]
                           ] 
                         )


Remember that the only feature we are using is the input $x$, the angle for which we want to find $sin(x)$.

In [12]:
def state_preparation(features2):
    for i, state in enumerate(features2):
        if state == 1.:
            qml.X(i)

def layer(layer_weights, num_qubits):
    for i in range(0, num_qubits):
        qml.Rot(*layer_weights[i], wires=i)
    for i in range(0, num_qubits - 1):
        qml.CNOT([i, (i + 1)])

@qml.qnode(dev)
def circuit(weights, feature, num_qubits):
    state_preparation(feature)
    for layer_weights in weights:
        layer(layer_weights, num_qubits)
    return np.array([qml.expval(qml.PauliZ(i)) for i in range(num_qubits)])


We train a model to get the answer we want. `cost` needs some modifications. We change `square_loss` to find the difference between `label` and the solution produced by the model. `cost` gives weight to each input based on their position in the list, with the first entries having higher cost.

In [13]:
def square_loss(labels, predictions_bits):
    # We give more weight to the first entries
    return sum([ 
        sum(
            [ (l - p) ** 2 / (i + 1)   for i, (l,p) in enumerate(zip(label, prediction))]
            )  
                for label, prediction in zip(labels, predictions_bits)
                ]) / len(labels)
    
def variational_classifier(weights, bias, feature, num_qubits):
    return circuit(weights, feature, num_qubits) + bias

def cost(weights, bias, features, labels, num_qubits):
    return square_loss(labels, 
                       np.array([variational_classifier(weights, bias, feature, num_qubits) 
                                 for feature in features], requires_grad=True))
def accuracy(predictions, labels):
    l = sum(1 if np.all(np.sign(np.array(p)) == np.array(l)) else 0 for (p, l) in zip(predictions, labels))   
    return l / len(predictions) * 100

Something interesting happens in `square_loss`. `bits` are expectation values $\in[-1,1]$. We don't round them! 

In [14]:
num_layers = 4
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.5, requires_grad=True)
opt = NesterovMomentumOptimizer(0.1)
batch_size = 10
weights = weights_init
bias = bias_init
for it in range(250):
    # Update the weights by one optimizer step, using only a limited batch of data
    batch_index = np.random.randint(0, len(features_train2), (batch_size,))
    features_batch = features_train2[batch_index]
    labels_batch = labels_train2[batch_index]
    weights, bias = opt.step(cost, weights, bias, features=features_batch, labels=labels_batch, num_qubits=num_qubits)
    current_cost = cost(weights, bias, features_train2, labels_train2, num_qubits)
    # Compute accuracy
    predictions = [(variational_classifier(weights, bias, x, num_qubits)) for x in features_train2]
    acc = accuracy(predictions, labels_train2)
    print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Acc: {acc:0.5f} %")

Iter:    1 | Cost: 8.2067449 | Acc: 0.00000 %
Iter:    2 | Cost: 8.1761092 | Acc: 0.00000 %
Iter:    3 | Cost: 8.1248863 | Acc: 0.00000 %
Iter:    4 | Cost: 7.9126442 | Acc: 0.00000 %
Iter:    5 | Cost: 6.9182861 | Acc: 0.00000 %
Iter:    6 | Cost: 4.0544131 | Acc: 0.00000 %
Iter:    7 | Cost: 1.6396671 | Acc: 0.00000 %
Iter:    8 | Cost: 1.0811020 | Acc: 47.50000 %
Iter:    9 | Cost: 1.1297739 | Acc: 47.50000 %
Iter:   10 | Cost: 1.1665300 | Acc: 47.50000 %
Iter:   11 | Cost: 1.1119887 | Acc: 26.25000 %
Iter:   12 | Cost: 0.9949251 | Acc: 39.58333 %
Iter:   13 | Cost: 0.8837914 | Acc: 53.33333 %
Iter:   14 | Cost: 0.8346043 | Acc: 53.33333 %
Iter:   15 | Cost: 0.8261692 | Acc: 40.41667 %
Iter:   16 | Cost: 0.8326259 | Acc: 40.41667 %
Iter:   17 | Cost: 0.8185416 | Acc: 40.41667 %
Iter:   18 | Cost: 0.7875008 | Acc: 40.41667 %
Iter:   19 | Cost: 0.7445721 | Acc: 40.41667 %
Iter:   20 | Cost: 0.7088951 | Acc: 40.41667 %
Iter:   21 | Cost: 0.6781153 | Acc: 36.66667 %
Iter:   22 | Cost: 0

To clarify the meaning of the output here, it is not about expectation values like in the previous model, but about what is more likely, a zero or a one. We run the model multiple times, if a qubit has higher probability to be one, then this qubit's output is one. The benefit of this approach is that we can achieve perfect accuracy without achieving 0 cost. Though, 0 cost is of course better and means one shot is enough to get the correct answer.

In [15]:
predictions = [(variational_classifier(weights, bias, x,num_qubits)) for x in features_test2]
predictions = [np.array([int(np.sign(bit)) for bit in prediction]) for prediction in predictions]

print("Accuracy on test data:", accuracy(predictions, labels_test2) , "%")

Accuracy on test data: 100.0 %


### Another way to approximate any function
If interested, you should check this Pennylane tutorial to approximate sin using fourier analysis
https://pennylane.ai/qml/demos/tutorial_expressivity_fourier_series/